Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Deploying a web service to Azure Kubernetes Service (AKS)
In this notebook, we show the following steps for deploying a web service using AzureML:
- Provision an AKS cluster (one time action)
- Deploy the service
- Test the web service
- Scale up the service

In [ ]:
import json
import os
import subprocess

import numpy as np
import pandas as pd
import requests
from MetricsUtils.hpStatisticsCollection import statisticsCollector, CollectionEntry
from azure_utils.machine_learning.utils import load_configuration
from azure_utils.utilities import text_to_json
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azure_utils.machine_learning.utils import get_workspace_from_config
from notebooks import DIRECTORY

AML will use the following information to create an image, provision a cluster and deploy a service. Replace the 
values in the following cell with your information.

In [ ]:
cfg = load_configuration("../workspace_conf.yml")

In [ ]:
image_name = cfg['image_name']
aks_service_name = cfg['aks_service_name']
aks_name = cfg['aks_name']
aks_location = cfg['workspace_region']

## Get workspace
Load existing workspace from the config file.

In [ ]:
ws = get_workspace_from_config()
print(ws.name, ws.resource_group, ws.location, sep="\n")

In [ ]:
image = ws.images[image_name]

Restore the statistics data.

## Provision the AKS Cluster
This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete 
the cluster or the resource group that contains it, then you would have to recreate it. Let's first check if there are 
enough cores in the subscription for the cluster .

In [ ]:
vm_family = "Dv2"
vm_size = "Standard_D4_v2"
vm_cores = 8
node_count = 4

In [ ]:
vm_dict = {vm_family: {"size": vm_size, "cores": vm_cores}}

In [ ]:
prov_config = AksCompute.provisioning_configuration(agent_count=node_count,
                                                    vm_size=vm_size,
                                                    location=aks_location)

# Create the cluster
aks_target = ComputeTarget.create(workspace=ws,
                                  name=aks_name,
                                  provisioning_configuration=prov_config)

In [ ]:
%%time
aks_target.wait_for_completion(show_output=True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Let's check that the cluster is created successfully.

In [ ]:
aks_status = aks_target.get_status()

In [ ]:
assert aks_status == 'Succeeded', 'AKS failed to create'

## Deploy web service to AKS

Next, we deploy the web service. We deploy two pods with 1 CPU core each.

In [ ]:
num_replicas = 2
cpu_cores = 1

In [ ]:
#Set the web service configuration
aks_config = AksWebservice.deploy_configuration(num_replicas=num_replicas,
                                                cpu_cores=cpu_cores)

In [ ]:
aks_service = Webservice.deploy_from_image(
    workspace=ws,
    name=aks_service_name,
    image=image,
    deployment_config=aks_config,
    deployment_target=aks_target,
)

In [ ]:
%%time
aks_service.wait_for_deployment(show_output=True)
print(aks_service.state)

You can check the logs of the web service with the below.

In [ ]:
aks_service.get_logs()

## Test the web service
We now test the web service.

In [ ]:
num_dupes_to_score = 4

In [ ]:
dupes_test_path = DIRECTORY + '/data_folder/dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
text_to_score = dupes_test.iloc[0, num_dupes_to_score]
text_to_score

In [ ]:
json_text = text_to_json(text_to_score)

In [ ]:
%%time
prediction = aks_service.run(input_data=json_text)
print(prediction)

Let's try a few more duplicate questions and display their top 3 original matches. Let's first get the scoring URL 
and API key for the web service.

In [ ]:
scoring_url = aks_service.scoring_uri
api_key = aks_service.get_keys()[0]

Write the URI and key to the statistics tracker.

In [ ]:
headers = {
    'content-type': 'application/json',
    'Authorization': ('Bearer ' + api_key)
}
r = requests.post(
    scoring_url, data=json_text,
    headers=headers)  # Run the request twice since the first time takes a
%time r = requests.post(scoring_url, data=jsontext, headers=headers) # little longer due to the loading of the model
print(r)
r.json()

In [ ]:
dupes_to_score = dupes_test.iloc[:5, num_dupes_to_score]

In [ ]:
results = [
    requests.post(scoring_url, data=text_to_json(text), headers=headers)
    for text in dupes_to_score
]

Let's print top 3 matches for each duplicate question.

In [ ]:
[eval(results[i].json())[0:3] for i in range(0, len(results))]

Next let's quickly check what the request response performance is for the deployed model on AKS cluster.

In [ ]:
text_data = list(map(text_to_json, dupes_to_score))  # Retrieve the text data

In [ ]:
timer_results = list()
for text in text_data:
    res=%timeit -r 1 -o -q requests.post(scoring_url, data=text, headers=headers)
    timer_results.append(res.best)

In [ ]:
timer_results

In [ ]:
print("Average time taken: {0:4.2f} ms".format(10 ** 3 * np.mean(timer_results)))

## Scaling

In this part, we scale the number of pods to make sure we fully utilize the AKS cluster. To connect to the Kubernetes 
cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [ ]:
!sudo az aks install-cli

Next, we will get the credentials to connect to the cluster.

In [ ]:
os.makedirs(os.path.join(os.path.expanduser('~'),'.kube'), exist_ok=True) 

In [ ]:
config_path = os.path.join(os.path.expanduser('~'),'.kube/config')

In [ ]:
with open(config_path, 'a') as f:
    f.write(aks_target.get_credentials()['userKubeConfig'])

Let's check the nodes and pods of the cluster.

In [ ]:
!kubectl get nodes

In [ ]:
!kubectl get pods --all-namespaces

In [ ]:
!kubectl get events

We can now scale up the number of pods.

In [ ]:
new_num_replicas = 10

In [ ]:
!kubectl get namespaces

In [ ]:
!kubectl scale --current-replicas=$num_replicas \
    --replicas=$new_num_replicas {"deployments/" + aks_service_name} \
    --namespace azureml-workspace

In [ ]:
!kubectl get pods --all-namespaces

In [ ]:
!kubectl get deployment

Next, we will test the [throughput of the web service](06_SpeedTestWebApp.ipynb).